In [ ]:
!pip install flask --break-system-packages
!pip install flask-restx  --break-system-packages
!pip install flask-sqlalchemy --break-system-packages
!pip install mysql-connector-python --break-system-packages
!pip install flask-migrate --break-system-packages

In [ ]:
#Flask-RESTx API Documentation Lesson
# Create a basic Flask app, initialize Flask-RESTx, define a namespace, and add resources

import nest_asyncio
from werkzeug.serving import run_simple
from flask import Flask
from flask_restx import Api, Namespace, Resource

app = Flask(__name__)
api = Api(app, title="Sample API", description="A sample API to demonstrate Flask-RESTx")

# Define a namespace :: Namespaces allow you to organize API endpoints.
# Namespace organizes resources logically and provides grouping for related endpoints.
# Each namespace can have a description and prefix.
hello_ns = Namespace('hello', description="Hello World operations")

# Define resources
# Use @namespace.route() to define routes within a namespace.
# Extend the Resource class to define HTTP methods like GET, POST, etc.
@hello_ns.route('/')
class HelloWorld(Resource):
    def get(self):
        """Returns a hello message"""  # The __doc__ string is used as the endpoint description in the documentation.
        return {"message": "Hello, World!"}

# Add the namespace to the API
api.add_namespace(hello_ns)

nest_asyncio.apply()
run_simple('localhost', 5000, app)

In [ ]:
# Using Models for Input and Output Validation
# Define data models using Flask-RESTx’s Model for input validation and documentation

import nest_asyncio
from werkzeug.serving import run_simple
from flask import Flask, jsonify, request
from flask_restx import Api, Namespace, Resource
from flask_restx import fields
 
app = Flask(__name__)
api = Api(app, title="Sample API", description="A sample API to demonstrate Flask-RESTx")

hello_model = hello_ns.model('Hello', {
    'name': fields.String(required=True, description='The name to greet'),
    'age': fields.Integer(required=False, description='Age of the person')
})

# Use fields to define the structure of your request and response.
# @namespace.model() creates a reusable model.
# @namespace.expect() specifies the expected input for an endpoint.
@hello_ns.route('/greet')
class Greet(Resource):
    @hello_ns.expect(hello_model)
    def post(self):
        """Greet a person by name"""
        data = api.payload  # Access JSON payload instead of using  request.get_json() api also gives a way of harvesting payload from request
        return {"message": f"Hello, {data['name']}!"}
    
# Add the namespace to the API
api.add_namespace(hello_ns)

nest_asyncio.apply()
run_simple('localhost', 5000, app)

In [ ]:
# Using Models for Input and Output Validation
# Define data models using Flask-RESTx’s Model for input validation and documentation
# Adding Responses

import nest_asyncio
from werkzeug.serving import run_simple
from flask import Flask, jsonify, request
from flask_restx import Api, Namespace, Resource
from flask_restx import fields
 
app = Flask(__name__)
api = Api(app, title="Sample API", description="A sample API to demonstrate Flask-RESTx")

hello_model = hello_ns.model('Hello', {
    'name': fields.String(required=True, description='The name to greet'),
    'age': fields.Integer(required=False, description='Age of the person')
})

# Use fields to define the structure of your request and response.
# @namespace.model() creates a reusable model.
# @namespace.expect() specifies the expected input for an endpoint.
@hello_ns.route('/greet')
@hello_ns.response(200, 'Success')
@hello_ns.response(400, 'Validation Error')
class Greet(Resource):
    @hello_ns.expect(hello_model)
    def post(self):
        """Greet a person by name"""
        data = api.payload  # Access JSON payload instead of using  request.get_json() api also gives a way of harvesting payload from request
        return {"message": f"Hello, {data['name']}!"}
    
# Add the namespace to the API
api.add_namespace(hello_ns)

nest_asyncio.apply()
run_simple('localhost', 5000, app)

In [ ]:
# Using Models for Input and Output Validation
# Define data models using Flask-RESTx’s Model for input validation and documentation
# Adding Responses
# Nesting Models

import nest_asyncio
from werkzeug.serving import run_simple
from flask import Flask, jsonify, request
from flask_restx import Api, Namespace, Resource
from flask_restx import fields
 
 # Define the Address model
address_model = hello_ns.model('Address', {
    'city': fields.String(required=True, description='City name'),
    'zip': fields.String(required=True, description='ZIP code')
})

# Define the User model
user_model = hello_ns.model('User', {
    'name': fields.String(required=True, description='User name'),
    'address': fields.Nested(address_model, description='User address')
})

# Define the Complex model
complex_model = hello_ns.model('Complex', {
    'user': fields.Nested(user_model, required=True, description='User details'),
    'metadata': fields.Raw(description='Additional data in any format')
})

 
@hello_ns.route('/complex')
class ComplexResource(Resource):
    @hello_ns.expect(complex_model)
    @hello_ns.marshal_with(complex_model)
    def post(self):
        """Create a complex resource"""
        example_data = {
            'user': {
                'name': 'John Doe',
                'address': {
                    'city': 'New York',
                    'zip': '10001'
                }
            },
            'metadata': {
                'info': 'Some additional data'
            }
        }
        
    
# Add the namespace to the API
api.add_namespace(hello_ns)

nest_asyncio.apply()
run_simple('localhost', 5000, app)

In [ ]:
# Using Models for Input and Output Validation
# Define data models using Flask-RESTx’s Model for input validation and documentation
# Adding Responses
# Implementing Query Parameters
import nest_asyncio
from werkzeug.serving import run_simple
from flask import Flask, jsonify, request
from flask_restx import Api, Namespace, Resource
from flask_restx import fields
from flask_restx import reqparse

app = Flask(__name__)
api = Api(app, title="Sample API", description="A sample API to demonstrate Flask-RESTx")

parser = hello_ns.parser()
parser.add_argument('name', type=str, required=True, help="Name of the person")
parser.add_argument('age', type=int, help="Age of the person")

@hello_ns.route('/query')
class QueryExample(Resource):
    @hello_ns.expect(parser)
    def get(self):
        """Example of query parameters"""
        args = parser.parse_args()
        return {"name": args['name'], "age": args.get('age', 'Not provided')}
    
# Add the namespace to the API
api.add_namespace(hello_ns)

nest_asyncio.apply()
run_simple('localhost', 5000, app)

In [ ]:
# complete example Complete Example
import nest_asyncio
from werkzeug.serving import run_simple
from flask import Flask
from flask_restx import Api, Namespace, Resource, fields

app = Flask(__name__)
api = Api(app, title="Sample API", version="1.0", description="A simple API")

# Define namespace
hello_ns = Namespace('hello', description="Hello World operations")

# Define models
address_model = hello_ns.model('Address', {
    'city': fields.String(required=True),
    'zip': fields.String(required=True)
})

user_model = hello_ns.model('User', {
    'name': fields.String(required=True),
    'address': fields.Nested(address_model)
})

complex_model = hello_ns.model('Complex', {
    'user': fields.Nested(user_model, required=True),
    'metadata': fields.Raw(description='Additional data in any format')
})

# Routes
@hello_ns.route('/')
class HelloWorld(Resource):
    def get(self):
        """Returns a hello message"""
        return {"message": "Hello, World!"}

@hello_ns.route('/greet')
class Greet(Resource):
    @hello_ns.expect(user_model)
    def post(self):
        """Greet a user"""
        data = api.payload
        return {"message": f"Hello, {data['name']} from {data['address']['city']}!"}

@hello_ns.route('/complex')
class ComplexExample(Resource):
    @hello_ns.expect(complex_model)
    def post(self):
        """Handles complex nested input"""
        data = api.payload
        return {"user": data['user'], "metadata": data['metadata']}

# Add the namespace to the API
api.add_namespace(hello_ns)

nest_asyncio.apply()
run_simple('localhost', 5000, app)